In [1]:
import vtk
from ipyvtklink.viewer import ViewInteractiveWidget

## VTK Pipeline

I created a class called `vtk_pipeline` to organize all the variables and function for rendering a 2D image from Dicom or Niftii file in one place. You can simply create an instance of this class and specify the path and type of the files as well as window and level that must be used for rendering the images. 

In [2]:
class vtk_pipeline:
    
    
    def __init__(self, path, filetype,window, level):
        self.path = path
        self.filetype = filetype
        self.window = window
        self.level = level
        
        self.out = None
        self.dim = None
        self.image_reader()
        
        self.mapper = None
        self.actor = None
        self.renderer = None
        self.render_window= None
        self.interactor = None
        
    def image_reader(self):   
        if self.filetype.lower() == 'dicom':  # Load a DICOM image from file into a vtk pipeline.
            img_reader = vtk.vtkDICOMImageReader()
            img_reader.SetDirectoryName(self.path)
            img_reader.Update()
            self.out = img_reader.GetOutput()
            self.dim = self.out.GetDimensions()
        elif self.filetype.lower() == 'niftii': # Load a NIFTI image from file into a vtk pipeline.
            img_reader = vtk.vtkNIFTIImageReader()
            img_reader.SetFileName(self.path)
            img_reader.Update()
            self.out = img_reader.GetOutput()
            self.dim = self.out.GetDimensions()

    def slice_navigator(self, obj, eid, *args, **kwargs): #callback function used for navigating through slices using mouse
        if eid == "MouseWheelForwardEvent":
            slice_num = self.mapper.GetZSlice()
            self.mapper.SetZSlice(slice_num+1)
        elif eid == "MouseWheelBackwardEvent":
            slice_num = self.mapper.GetZSlice()
            self.mapper.SetZSlice(slice_num-1)
            
            
    def render(self):
        self.mapper = vtk.vtkImageMapper()
        self.mapper.SetColorWindow(self.window) #the lower, the brighter the image would be (more contrast)
        self.mapper.SetColorLevel(self.level) #setting the level
        self.mapper.SetZSlice(150) #setting the default slice to show (change it later by observer)
        self.mapper.SetInputData(self.out)

        self.actor = vtk.vtkActor2D()
        self.actor.SetMapper(self.mapper)

        self.renderer = vtk.vtkRenderer()
        self.renderer.AddActor(self.actor) #Add the actor to the scene
        self.renderer.ResetCamera()



        self.render_window = vtk.vtkRenderWindow()
        self.render_window.SetOffScreenRendering(True)
        self.render_window.SetSize(512,512) #set a constant size for render window
        self.render_window.AddRenderer(self.renderer)

        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetRenderWindow(self.render_window)
        self.interactor.SetInteractorStyle(vtk.vtkInteractorStyleImage())
        self.interactor.AddObserver("MouseWheelForwardEvent", self.slice_navigator)
        self.interactor.AddObserver("MouseWheelBackwardEvent", self.slice_navigator)

In [3]:
hip_path = "/Users/nathanneeteson/Library/CloudStorage/OneDrive-UniversityofCalgary/MDSC68903_ImageData/Hip"
thrx_path = "/Users/nathanneeteson/Library/CloudStorage/OneDrive-UniversityofCalgary/MDSC68903_ImageData/Thorax"
head_path = "/Users/nathanneeteson/Library/CloudStorage/OneDrive-UniversityofCalgary/MDSC68903_ImageData/head.nii"

## Render Hip Dicom

In [4]:
hip = vtk_pipeline(path=hip_path,filetype='dicom',window=400,level=40)
hip.render()
ViewInteractiveWidget(hip.render_window)

ViewInteractiveWidget(height=512, layout=Layout(height='auto', width='100%'), width=512)

**2D slice render from Hip sample dataset** Start Slice: 150, Window: 400, Level: 40

## Render Thorax Dicom

In [5]:
thorax = vtk_pipeline(path=thrx_path, filetype='dicom', window=2000, level=40)

thorax.render()

ViewInteractiveWidget(thorax.render_window)

ViewInteractiveWidget(height=512, layout=Layout(height='auto', width='100%'), width=512)

**2D slice render from Thorax sample dataset** Start Slice: 150, Window: 2000, Level: 40

## Render Head Niftii

In [6]:
head = vtk_pipeline(path=head_path, filetype='niftii', window= 700, level= 250)
head.render()

ViewInteractiveWidget(head.render_window)

ViewInteractiveWidget(height=512, layout=Layout(height='auto', width='100%'), width=512)

**2D slice render from Hip sample dataset** Start Slice: 150, Window: 700, Level: 250